#            Implementation of Image Web Scraping using Selenium Python

This notebook was adapted from this [this github repository](https://github.com/Praveen76/Web-Scraping-using-Selenium-Python)

# Step1. Import all required libraries

In [1]:
#Import libraries
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException

# Step2. Install Chrome Driver

In [2]:
#Install Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\oelyd\.wdm\drivers\chromedriver\win32\88.0.4324.96]


# Step3. Specify Search url

In [3]:
#Specify Search URL
search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
driver.get(search_url.format(q='decathlon'))

 - I've used this specific URL so you don't get in trouble for using licensed or images with copyrights. Otherwise you can use https://google.com also as search URL .
 - Then we're searching for Car in our Search URL.

### Paste the link into to driver.get(“ Your Link Here ”) function and run the cell. This will open a new browser window for that link. 


# Step4. Scroll to the end of the page

In [4]:
#Scroll to the end of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)#sleep_between_interactions  


#### The above line of code would help us to reach the end of the page . And then we're giving sleep time of 5 seconds so we don't run in problem , where we're trying to read elements from page , which is not yet loaded.

# Step5. Locate the images to be scraped from the page

In [5]:
#Locate the images to be scraped from the current page
imgResults = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
totalResults=len(imgResults)

### Now we'll fetch all the image links present on that particular page. We will create a “list” to store those links.
So, to do that go to the browser window, right-click on the page, and select ‘inspect element’ or enable the dev tools using Ctrl+Shift+I .
Now identify any attribute such as class , id etc. Which is common across all these images.

In our case class ="'Q4LuWd" is common across all these images. 


# Step6. Extract corresponding link of each Image

In [6]:
#Click on each Image to extract its corresponding link to download

img_urls = set()
for i in  range(0,len(imgResults)):
    img=imgResults[i]
    try:
        img.click()
        time.sleep(2)
        actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
        for actual_image in actual_images:
            if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                img_urls.add(actual_image.get_attribute('src'))
    except ElementClickInterceptedException or ElementNotInteractableException as err:
        print(err)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=88.0.4324.190)



So in the above snippet of code , we're performing following tasks
   - Iterate through each thumbnail and then on click it .
   - Make our browser sleep for 2 seconds (:P) .
   - Find the unique html tag corresponding to that image to locate it on page
   - We still get more than one result for particular image .  But all we're interested in the link for that image to download.

So we iterate through each result for that image and extract 'src' attribute of it and then see whether "https" is present in the 'src'  or not. Since typically web link starts with 'https'.

# Step7. Download & save each image in Destination directory

In [7]:
os.chdir('C:/Users/oelyd/Desktop/image_processing_side_projects/scraping_decathlon')
baseDir=os.getcwd()

for i, url in enumerate(img_urls):
    file_name = f"{i:150}.jpg"    
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(baseDir, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQUFXZCiy2Kv4JQFDjrpFHaMshaY91rBl169A&usqp=CAU - AT: C:\Users\oelyd\Desktop\image_processing_side_projects\scraping_decathlon\                                                                                                                                                     0.jpg
SAVED - https://upload.wikimedia.org/wikipedia/commons/0/00/Decathlon_Store_in_China.jpg - AT: C:\Users\oelyd\Desktop\image_processing_side_projects\scraping_decathlon\                                                                                                                                                     1.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT5drFiGabR0WxlS9O7HJHoX6KUF_R0jw_5yg&usqp=CAU - AT: C:\Users\oelyd\Desktop\image_processing_side_projects\scraping_decathlon\                                                                                                                                                    